<a href="https://colab.research.google.com/github/Prashanth2902/PrashanthPrabhakar/blob/master/token_counts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Empirical Regularities of Language

In this first homework assignment, you will familiarize yourself with some empirical regularities of natural language, Shannon entropy and Zipf's Law.

Read through this Jupyter notebook and fill in the parts marked with `TODO`. When you're ready to submit, print the notebook as a PDF and upload to Gradescope.

## Shannon Entropy

Shannon borrowed the concept of entropy from statistical physics to develop _information theory_, focused on encoding and compressing messages. A few years later, in 1950, he applied information theory to analyze human predictive ability—in other words, the entropy of the human language model. You can read the original article, [Prediction and Entropy of Printed English](https://languagelog.ldc.upenn.edu/myl/Shannon1950.pdf), for more details.

Your first task is to collect data on how predictable different letters are in an English sentence, depending on how much context in a word or sentence you have.

Go to the [Shannon game page](https://www.ccs.neu.edu/home/dasmith/courses/cs6120/shannon/) that we demonstrated in class. We already guessed part of Text 1, so work through Texts 2, 3, and 4.

In [ ]:
# TODO: Enter the arrays of numbers of guesses for Texts 2, 3, and 4 here.
import numpy as np
import pandas as pd

word1 = [17, 1, 13, 1, 1, 1, 1, 1, 26, 1, 1, 1, 1, 2, 1, 1, 1, 1, 24, 1, 19, 4, 1, 1, 3, 1, 1, 1, 3, 2, 10, 1, 6, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1],
word2 = [1, 3, 1, 7, 4, 1, 3, 1, 11, 4, 8, 10, 7, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 11, 1, 1, 1, 1, 1, 1, 1, 1, 1, 24, 1, 25, 2, 1, 1, 7, 9, 10, 23, 1, 15, 1, 3, 15, 1, 27, 25, 6, 1, 17, 1, 18, 8, 6, 1, 1, 1, 14, 3, 13, 1]
word3 = [8, 2, 8, 1, 1, 25, 14, 1, 1, 1, 1, 1, 1, 1, 1, 15, 2, 1, 1, 9, 1, 1, 1, 1, 16, 1, 5, 9, 13, 3, 1, 1, 16, 1, 14, 1, 1, 1, 1, 10, 25, 26, 18, 1, 14, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Rearrange the guess data into a two-dimensional array, relating number of characters of context (0, 1, 2, ...) to number of guesses required.

In other words, you might look in cell (2, 1) and read "2" if the number of times it took one guess to get the right letter with two characters of context was 2.

In [25]:
# TODO: Create array of counts of guesses. Print out the array so we can see it.
def counts_1_to_27(xs):
    c = [0]*27
    for g in xs:
        c[g-1] += 1
    return c

word1_counts = counts_1_to_27(word1)
word2_counts = counts_1_to_27(word2)
word3_counts = counts_1_to_27(word3)

guess_array = np.array([word1_counts, word2_counts, word3_counts])
print("Output 2D Array\n")
print(guess_array)

Output 2D Array

[[33  4  2  1  0  1  0  0  0  1  0  0  1  0  0  0  1  0  1  0  0  0  0  1
   0  1  0]
 [35  2  5  2  0  2  3  2  1  2  2  0  1  1  2  0  1  1  0  0  0  0  1  1
   2  0  1]
 [36  2  1  0  2  0  0  2  2  1  0  0  1  3  1  2  0  1  0  0  0  0  0  0
   2  1  0]]


Now you can compute Shannon's upper and lower bounds on the entropy of your predictive distribution for English. The upper bound, as a function of the number of context characters $N$, is just the Shannon entropy of the distribution of numbers of guesses. In other words, it's the entropy of the original text as &ldquo;reduced&rdquo; by the human encoder to a sequence of numbers of guesses.

$F_N = -\sum_{i=1}^{27} q_i^N \log_2 q_i^N$

where $q_i^N$ is the number of times you took $i$ guesses with $N$ characters of context, i.e., one of the cells in the table you computed. The number of guesses ranges from 1 to 27 since we restrict ourselves to 26 letters plus space. In computing entropy, we define $0 \log 0 = 0$.

In [26]:
# TODO: Compute the upper bound for each amount of context N and print it out.

totals = guess_array.sum(axis=0)         # add up counts across all rows
total_sum = totals.sum()                 # total number of guesses
probs = totals / total_sum               # probabilities
entropy_upper = -np.sum(probs[probs > 0] * np.log2(probs[probs > 0]))

print("Upper bound entropy:", entropy_upper)

Upper bound entropy: 2.626185522490259


Shannon derived a lower bound on entropy from the guess data as

$\sum_{i=1}^{27} i(q_i^N - q_{i+1}^N) \log_2 i$

In [28]:
# TODO: Compute thew lower bound for each amount of context N and print it out.

lower_bound = 0.0
for i in range(1, 28):             # i = 1..27
    q_i   = totals[i-1]
    q_ip1 = totals[i] if i < 27 else 0
    diff  = q_i - q_ip1
    if diff > 0:
        lower_bound += i * diff * np.log2(i)

print("Lower bound entropy:", lower_bound)

Lower bound entropy: 948.7823003617274


## Zipf's Law

Now let's look at some text data directly to see the skewed distribution of tokens predicted by Zipf's Law. Recall that Zipf's law states that a word's rank (from the most common word at rank 1 on down) to its frequency is approximately a constant, i.e., $r \cdot f = k$. Equivalently, we can divide both sides by the total number of tokens $N$ to get $r \cdot P_r = c$, where $c = k/N$ and $P_r = f/N$ is the _relative frequency_ of word $r$.

We start by downloading a sample of 1000 open-access English books from [Project Gutenberg](https://gutenberg.org/).

In [ ]:
# If your local environment doesn't have the wget command,
# you can comment this out and download it manually.
!wget "http://khoury.northeastern.edu/home/dasmith/pg-sample.json.gz"

The file is compressed with gzip and is in a JSON lines format. Each line is one JSON record, which we parse with the `json` library.

Here we print out the keys in the first record: `id`, `author`, `title`, and `text`.

In [ ]:
import gzip, json
for line in gzip.open("pg-sample.json.gz", mode="rt", encoding="utf-8"):
  rec = json.loads(line)
  print(rec.keys())
  print(rec['author'])
  print(rec['title'])
  print(rec['text'][0:100])
  break

dict_keys(['id', 'author', 'title', 'text'])
Jefferson, Thomas
The Declaration of Independence of the United States of America


This is a retranscription of one of the first Project
Gutenberg Etexts, offically dated December 3


Your task now is to **tokenize** the text in the `text` field of each record into an array of words. Later on in this course, we will discuss learning better tokenizers. For now, you should separate words on whitespace (space, newline, tab) and punctuation. Convert the tokens to lower case, and keep only those tokens that have at least one letter a-z in them. In general, numerals in text tend not to follow Zipf's law but [Benford's law](https://en.wikipedia.org/wiki/Benford%27s_law).

You might use _regular expressions_ (e.g., the `re.split` function) to help with tokenization and filtering.

After you have tokenized, compute $N$, the total number of tokens in the corpus and print it out.

In [ ]:
# TODO: Compute an array of tokens in the corpus
# Compute the total number of tokens N and print it out.

Now, count the frequency each unigram (distinct word) in the corpus and sort them in an array in descending order of frequency. The first item in your array should be the most common word. Print out that word and its frequency

In [ ]:
# TODO: Compute an arrary of unigrams in descending order of frequency.
# Print the most common word and its frequency.

Now, you can look at the Zipf's law relationship between rank and relative frequency (i.e., frequency divided by $N$). Plot the data using a python graphing package such as matplotlib, plotly, or plotnine. This doesn't have to be a fancy graph, so use whatever you're familiar with. Both axes should be on a log scale. If your package doesn't support log scales, you can take the log of the rank and relative frequency yourself before plotting. Recall that since python arrays are zero-indexed, the rank 1 word will be element 0 of your sorted array.

In [ ]:
# TODO: Plot rank vs. relative frequency of unigrams.

Now, take your array of tokens and compute the counts of both the bigrams and trigrams and sort them in descending order of frequency. Print out the most common bigram and trigram.

In [ ]:
# TODO: Compute sorted bigram and trigram statistics.
# Print out the most common bigram and trigram.
# Plot rank vs. relative frequency for bigrams and trigrams.
# You may make separate plots or put them on the same plot and label them.

**TODO**: Finally, write your visual impressions of the fit of the unigram, bigram, and trigram distributions. This doesn't need to be statistically rigorous.